In [1]:
import os
import time
import datetime
import requests
import pandas as pd
import pyarrow
import numpy as np
from prefect import flow, task # Prefect flow and task decorators
import time
from datetime import datetime, timezone
from zoneinfo import ZoneInfo

In [3]:
#read firms_output parquet
df_firms = pd.read_parquet("firms_outout", engine="pyarrow")
df_firms.info()
df_firms.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7557 entries, 0 to 7556
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype                       
---  ------           --------------  -----                       
 0   country_id       7557 non-null   object                      
 1   latitude         7557 non-null   float64                     
 2   longitude        7557 non-null   float64                     
 3   brightness       7557 non-null   float64                     
 4   scan             7557 non-null   float64                     
 5   track            7557 non-null   float64                     
 6   acq_date         7557 non-null   object                      
 7   acq_time         7557 non-null   object                      
 8   satellite        7557 non-null   object                      
 9   instrument       7557 non-null   object                      
 10  confidence       7557 non-null   int64                       
 11  version          

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,bright_t31,frp,daynight,acq_datetime,acq_datetime_th,acq_year,acq_month,acq_day,acq_hour,acq_minute
0,THA,19.20018,98.41888,313.73,1.61,1.25,2025-03-18,03:25:00,Terra,MODIS,...,300.65,11.43,D,2025-03-18 03:25:00,2025-03-18 10:25:00+07:00,2025,3,18,10,25
1,THA,19.50270,99.01566,313.63,1.76,1.30,2025-03-18,03:25:00,Terra,MODIS,...,297.01,15.15,D,2025-03-18 03:25:00,2025-03-18 10:25:00+07:00,2025,3,18,10,25
2,THA,19.50479,99.00983,310.06,1.76,1.30,2025-03-18,03:25:00,Terra,MODIS,...,297.13,9.46,D,2025-03-18 03:25:00,2025-03-18 10:25:00+07:00,2025,3,18,10,25
3,THA,14.02202,101.49030,314.13,3.51,1.76,2025-03-18,03:28:00,Terra,MODIS,...,295.19,36.21,D,2025-03-18 03:28:00,2025-03-18 10:28:00+07:00,2025,3,18,10,28
4,THA,14.04078,101.42501,321.71,1.25,1.11,2025-03-18,07:52:00,Aqua,MODIS,...,303.28,9.10,D,2025-03-18 07:52:00,2025-03-18 14:52:00+07:00,2025,3,18,14,52


In [4]:
#read weather_output parquet
df_weather = pd.read_parquet("weather_outout", engine="pyarrow")
df_weather.info()
df_weather.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2772 entries, 0 to 2771
Data columns (total 39 columns):
 #   Column               Non-Null Count  Dtype                       
---  ------               --------------  -----                       
 0   coord.lon            2772 non-null   float64                     
 1   coord.lat            2772 non-null   float64                     
 2   weather.id           2772 non-null   int64                       
 3   weather.main         2772 non-null   object                      
 4   weather.description  2772 non-null   object                      
 5   weather.icon         2772 non-null   object                      
 6   base                 2772 non-null   object                      
 7   main.temp            2772 non-null   float64                     
 8   main.feels_like      2772 non-null   float64                     
 9   main.pressure        2772 non-null   int64                       
 10  main.humidity        2772 non-null  

,coord.lon,coord.lat,weather.id,weather.main,weather.description,weather.icon,base,main.temp,main.feels_like,main.pressure,...,sys.sunset,timezone,id,name,cod,acq_year,acq_month,acq_day,acq_hour,acq_minute
0,100.5241,13.7279,803,Clouds,broken clouds,04n,stations,302.50,307.95,1004,...,1743161353,25200,1619278,Bangrak,200,2025,3,28,1,6
1,98.9063,8.0863,804,Clouds,overcast clouds,04n,stations,298.67,299.47,1007,...,1743161665,25200,1152633,Krabi,200,2025,3,28,1,6
2,99.5328,14.0228,802,Clouds,scattered clouds,03n,stations,301.23,302.61,1004,...,1743161595,25200,1157835,Ban Don Rak,200,2025,3,28,1,6
3,103.5059,16.4314,804,Clouds,overcast clouds,04n,stations,304.73,305.54,1002,...,1743160676,25200,1610469,Kalasin,200,2025,3,28,1,6
4,99.5227,16.4828,802,Clouds,scattered clouds,03n,stations,299.77,299.77,1003,...,1743161633,25200,1153090,Kamphaeng Phet,200,2025,3,28,1,6


In [14]:
df_weather = df_weather.rename(columns={'coord.lat': 'latitude', 'coord.lon': 'longitude'})

In [11]:
import pandas as pd
import numpy as np

from sklearn.neighbors import BallTree
from io import StringIO

import matplotlib.pyplot as plt
import contextily as ctx

In [21]:
# Convert coordinates to radians
firms_coords = np.deg2rad(df_firms[['latitude', 'longitude']])
weather_coords = np.deg2rad(df_weather[['latitude', 'longitude']])

# Create a BallTree for nearest neighbor search
tree = BallTree(weather_coords, metric='haversine')

# Find the nearest neighbors in OSM data
distances, indices = tree.query(firms_coords, k=1)

test_df = pd.DataFrame()
# Add the results to the DataFrame
df_firms['nearest_index'] = indices.flatten()
df_firms['distance_meters'] = distances.flatten() * 6371000 # Earth's radius in meters

# Extract and assign the matched OSM node details
df_firms[[
    'near_lat',
    'near_lon',
]] = df_weather.iloc[df_firms['nearest_index']][[
    'latitude',
    'longitude',
]].values

# test_df['node_id'] = test_df['node_id'].apply(lambda x: f'{int(x)}')
# test_df = test_df.drop('nearest_index', axis=1)

df_firms.head(5)

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,acq_datetime_th,acq_year,acq_month,acq_day,acq_hour,acq_minute,nearest_index,distance_meters,near_lat,near_lon
0,THA,14.95620,105.52222,322.12,1.76,1.30,2025-04-21,07:05:00,Aqua,MODIS,...,2025-04-21 14:05:00+07:00,2025,4,21,14,5,1156,77335.587004,15.2274,104.8589
1,THA,15.61316,104.78119,320.65,1.94,1.36,2025-04-21,07:05:00,Aqua,MODIS,...,2025-04-21 14:05:00+07:00,2025,4,21,14,5,71,32636.365532,15.8657,104.6258
2,THA,15.61592,104.76965,323.57,1.95,1.36,2025-04-21,07:05:00,Aqua,MODIS,...,2025-04-21 14:05:00+07:00,2025,4,21,14,5,71,31755.822843,15.8657,104.6258
3,THA,16.24913,100.47689,319.83,4.05,1.87,2025-04-21,07:07:00,Aqua,MODIS,...,2025-04-21 14:07:00+07:00,2025,4,21,14,7,577,26467.166585,16.4515,100.3463
4,THA,16.11451,102.25852,317.55,1.16,1.07,2025-04-21,02:56:00,Terra,MODIS,...,2025-04-21 09:56:00+07:00,2025,4,21,9,56,10,41949.596788,15.8068,102.0315


In [23]:
df_firms = df_firms.sort_values('acq_datetime_th')
df_weather = df_weather.sort_values('dt')

In [24]:
dfmerge = pd.merge_asof(df_firms, df_weather,
...                         left_on='acq_datetime_th' , right_on='dt',
                            left_by=['near_lat','near_lon'], right_by=['latitude','longitude'],
                            direction='nearest')

In [27]:
dfmerge.info()
dfmerge.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 613 entries, 0 to 612
Data columns (total 65 columns):
 #   Column               Non-Null Count  Dtype                       
---  ------               --------------  -----                       
 0   country_id           613 non-null    object                      
 1   latitude_x           613 non-null    float64                     
 2   longitude_x          613 non-null    float64                     
 3   brightness           613 non-null    float64                     
 4   scan                 613 non-null    float64                     
 5   track                613 non-null    float64                     
 6   acq_date_x           613 non-null    object                      
 7   acq_time_x           613 non-null    object                      
 8   satellite            613 non-null    object                      
 9   instrument           613 non-null    object                      
 10  confidence           613 non-null    i

,country_id,latitude_x,longitude_x,brightness,scan,track,acq_date_x,acq_time_x,satellite,instrument,...,sys.sunset,timezone,id,name,cod,acq_year_y,acq_month_y,acq_day_y,acq_hour_y,acq_minute_y
0,THA,16.11451,102.25852,317.55,1.16,1.07,2025-04-21,02:56:00,Terra,MODIS,...,1745234853,25200,1611407,Chaiyaphum,200,2025,4,21,22,15
1,THA,16.33279,99.68369,322.92,1.00,1.00,2025-04-21,02:56:00,Terra,MODIS,...,1745235493,25200,1153090,Kamphaeng Phet,200,2025,4,21,22,15
2,THA,17.98666,100.13609,314.77,1.00,1.00,2025-04-21,02:56:00,Terra,MODIS,...,1745235438,25200,1607552,Phrae,200,2025,4,21,22,15
3,THA,18.93324,98.27450,314.18,1.13,1.06,2025-04-21,02:56:00,Terra,MODIS,...,1745236027,25200,1152222,Mae Hong Son,200,2025,4,21,22,15
4,THA,18.93456,98.26445,315.09,1.13,1.06,2025-04-21,02:56:00,Terra,MODIS,...,1745236027,25200,1152222,Mae Hong Son,200,2025,4,21,22,15


In [28]:
dfmerge[['latitude_x','longitude_x','near_lat','near_lon','latitude_y','longitude_y']]

,latitude_x,longitude_x,near_lat,near_lon,latitude_y,longitude_y
0,16.11451,102.25852,15.8068,102.0315,15.8068,102.0315
1,16.33279,99.68369,16.4828,99.5227,16.4828,99.5227
2,17.98666,100.13609,18.1446,100.1403,18.1446,100.1403
3,18.93324,98.27450,19.3020,97.9654,19.3020,97.9654
4,18.93456,98.26445,19.3020,97.9654,19.3020,97.9654
...,...,...,...,...,...,...
608,15.49447,104.97675,15.2284,104.8586,15.2284,104.8586
609,15.60570,105.25562,15.2284,104.8586,15.2284,104.8586
610,15.24393,104.56009,15.1182,104.3220,15.1182,104.3220
611,15.40026,104.56705,15.2287,104.8564,15.2287,104.8564
